Origin : http://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html  
Translator : Hongpu Liu

# 神经网络

神经网络可以用**torch.nn**包来创建。

**nn**依赖于**autograd**，以定义模型及其微分。一个**nn.Module**包括了层、一个**forward(input)**方法，并返回其**output**。

例如，下面用来进行手写数字图像分类的网络：

![](./imgs/mnist.png)  

这是一个简单的前馈网络，它获得输入，将其前馈到后面的数个层中，最后获得输出。

典型的神经网络训练过程如下：

- 定义包含一些可学习参数（或权重）的神经网络
- 从数据集中迭代的获取输入
- 用神经网络计算输出
- 计算损失（输出与正确的标签之间的差异）
- 反向传播网络参数的梯度
- 更新网络的权重，一个典型、简单的更新规则：**weight = weight - learning_rate * gradient**

## 1. 定义神经网络
定义神经网络：

In [1]:
from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 输入图像1个通道，输出6个通道，采用5*5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 输入Feature Map有6通道，输出16个通道，采用5*5卷积核
        self.conv2 = nn.Conv2d(6,16,5)
        # 定义仿射变换：y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # 最大池化层使用一个 (2, 2)的窗口
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果池化层用的卷积核是方的，可以只写一个数值
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        # 注意输入的第0个维度是batch，因此每个样本的数量要去掉第0个维度。
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


由于使用了**autograd**，仅需要定义**forward**函数，**backward**函数（计算梯度的函数）会自动定义。在**forward**函数中可以进行任意的张量运算。

模型的可学习参数可以通过**net.parameters()**来返回。

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1层的权重

10
torch.Size([6, 1, 5, 5])


**forward**的输入和输出都是一个**autograd.Variable**。

>**注意**：该网络（LeNet）的输入是一个32x32的张量，在MNIST数据集上使用该网络，需要将数据集中的图像缩放为32x32。

In [3]:
# 第0个维度是batch，第1个维度是channel，第2、3个维度是图像的高和宽
input = Variable(torch.randn(1,1,32,32))
out = net(input)
print(out)

Variable containing:
-0.1058  0.1141  0.0983  0.0175  0.0834  0.1061 -0.0266  0.0616 -0.1618  0.1151
[torch.FloatTensor of size 1x10]



将梯度缓冲清零（每次**backward**都对累加梯度，因此需要清零），并传播某个随机梯度：

In [4]:
net.zero_grad()

# 输入的随机梯度第0个维度是batch，第1个维度是最后一层输出，out的维度是（1，10）
out.backward(torch.randn(1,10))

>**注意**

>**torch.nn**只支持小批量（mini-batch）作为输入。整个**torch.nn**只支持输入一组（mini-batch）的样本，而不支持输入一个样本。

>例如，**nn.Conv2d**需要输入一个4D张量：**nSamples x nChannels x Height x Width**。

>如果只有一个单独的样本，可以用**input.unsqueeze(0)**来添加一个假的batch维度。

在继续之前，回顾一下之前看到的所有类。

**回顾**  
- **torch.Tensor** - 多维数组
- **autograd.Variable** - 封装张量，并记录对该张量的所有历史操作。除了增加了一些如**backward()**之类的方法之外，与**Tensor**具有相同的API。此外还要保存对该张量求得的梯度。
- **nn.Module** - 神经网络模块。可以非常方便的封装参数，并实现将参数转到GPU、输出和加载等功能。
- **nn.Parameter** - 一种**Variable**，将其指定为一个**Module**的属性时，会自动注册为参数。
- **autograd.Function** - 实现了自动梯度计算中前馈与反馈的定义。每个**Variable**的运算，至少创建一个**Function**节点，连接到该运算所创建的**Variable**，并对其历史进行编码。

**目前已经讨论的内容**
- 定义一个神经网络
- 处理输出和调用反馈

**剩余的内容**
- 计算损失
- 更新网络权重

## 2. 损失函数（Loss Function）
损失函数以输入对应的*(output, target)*作为参数，计算一个输出与目标之间差异的估计值。

在**nn**包中有几个不同的损失函数。一个简单的损失函数为**nn.MSELoss**，该函数计算输入和目标之间的平均平方误差。

例如：

In [5]:
output = net(input)
target = Variable(torch.arange(1,11)) # 随便选的一个目标（target），仅用于举例
target = target.view(1, -1) # 将target的形状改为与output相同
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.2010
[torch.FloatTensor of size 1]



如果沿着**loss**的反方向，利用它的**.grad_fn**属性，就可以看到如下所示的计算图：

>    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  
>          -> view -> linear -> relu -> linear -> relu -> linear  
>          -> MSELoss  
>          -> loss  

因此，当调用**loss.backward()**时，计算损失对图中所有变量的微分，并将其累加到所有变量的**.grad**当中。

下面的代码将展示部分**backward**过程。

In [6]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

## 3. 反向传播（Backprop）
要反向传播误差，唯一要做的就是执行**loss.backward()**。由于该操作会累加梯度信息，因此在此之前需要清空现存的梯度。

接下来将要调用**loss.backward()**，并显示执行前后conv1层bias的梯度的差异。

In [7]:
net.zero_grad() # 将所有参数的梯度缓存清零

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
-0.0443
-0.0152
-0.0213
-0.0105
-0.1136
 0.0712
[torch.FloatTensor of size 6]



现在，我们已经看到了如何使用损失函数。

>**扩展阅读**  
>在神经网络包中，包含各种不同用于构建深度神经网络的模块和损失函数，完整的列表请参考[这里](http://pytorch.org/docs/nn)。


**还需要学习的内容**  
- 更新网络的权重

## 4. 更新权重
在实践中最简单的更新算法是采用随机梯度下降（Stochastic Gradient Descent, SGD）:

**weight = weight - learning_rate * gradient**

可以用非常简单的Python代码来实现：

``` Python
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
```

然而，在使用神经网络时，经常希望使用不同的更新规则，例如SGD、Nesterov-SGD、Adam、RMSProp等。包**torch.optim**实现了所有这些方法，非常易于使用：

In [8]:
import torch.optim as optim

# 创建一个优化器
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 在训练循环中可以用下面的方法完成训练
optimizer.zero_grad() # 清除梯度缓存
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

>**注意**  
>在上面的代码中，对梯度的缓存用**optimizer.zero_grad()**进行了手动清除。这是因为在反向传播的时候梯度是要累加的。